In [1]:
import sys
import requests
import json
import datetime
import os
import pandas as pd
import numpy as np
import os
from dateutil.relativedelta import relativedelta
import pytz
import json
import math
import ast
from sys import stdout as out

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


In [4]:
def get_dev_info(device, address):
    
    r = requests.post(address + "/api/auth/login",
                      json={'username': 'meazonpro@meazon.com', 'password': 'meazonpro1'}).json()
    
    # acc_token is the token to be used in the next request
    acc_token = 'Bearer' + ' ' + r['token']
    
    # get devid by serial name
    r1 = requests.get(
        url=address + "/api/tenant/devices?deviceName=" + device,
        headers={'Content-Type': 'application/json', 'Accept': '*/*', 'X-Authorization': acc_token}).json()
    print(r1)
    label = r1['label']
    devid = r1['id']['id']
    r1 = requests.get(
        url=address + "/api/device/" + devid + "/credentials",
        headers={'Content-Type': 'application/json', 'Accept': '*/*', 'X-Authorization': acc_token}).json()
    devtoken = r1['credentialsId']

    
    
    return devid,devtoken,acc_token,label
    



def read_data(acc_token, devid, address, start_time, end_time, descriptors):

        
    r2 = requests.get(
        url=address + "/api/plugins/telemetry/DEVICE/" + devid + "/values/timeseries?keys=" + descriptors + "&startTs=" + start_time + "&endTs=" + end_time + "&agg=AVERAGE,interval=60000&limit=1000000",
        headers={'Content-Type': 'application/json', 'Accept': '*/*', 'X-Authorization': acc_token}).json()
    
    if r2:
        df = pd.DataFrame([])
        
        for desc in r2.keys():
            df1 = pd.DataFrame(r2[desc])
            df1.set_index('ts', inplace=True)
            df1.columns = [str(desc)]
            
            df1.reset_index(drop=False, inplace=True)
            df1['ts'] = pd.to_datetime(df1['ts'], unit='ms')
            df1['ts'] = df1['ts'].dt.tz_localize('utc').dt.tz_convert('Europe/Athens')
            df1 = df1.sort_values(by=['ts'])
            df1.reset_index(drop=True, inplace=True)
            df1.set_index('ts', inplace=True, drop=True)            
            
            df = pd.concat([df, df1], axis=1)

        if df.empty:
            df = pd.DataFrame([])
        else:
            for col in df.columns:
                df[col] = df[col].astype('float64')
    else:
        df = pd.DataFrame([])
        # print('Empty json!')
    return df


In [ ]:
address = 'http://devmeazonthings.westeurope.cloudapp.azure.com:8080'

r = requests.post(address + "/api/auth/login",
                      json={'username': 'tenant@thingsboard.org', 'password': 'tenant'}).json()
    
acc_token = 'Bearer' + ' ' + r['token']



device = '102.408.000045'

# Define variables
# device = '102.408.000008'
local_tz = pytz.timezone('Europe/Athens')
interval = 1
descriptors = 'curA,curB,curC,cnrgA,cnrgB,cnrgC,cosA,cosB,cosC,efrqA,efrqB,efrqC,,vltA,vltB,vltC,kfac'
start_time = '1706824800000'
end_time = '1707818400000'

# extract info
[devid, _, acc_token,label] = get_dev_info(device, address)


nominal = get_attr(acc_token,devid,address)


# read data
timethres = 12*3600000
svec = np.arange(int(start_time),int(end_time),timethres)

df = pd.DataFrame([])

for st in svec:
    en = st+timethres-1
    
    if int(end_time)-en<=0: en = int(end_time)
    tmp = read_data(acc_token, devid, address,  str(st), str(en), descriptors)
    if not tmp.empty:
        tmp = tmp.resample(str(interval)+'T').mean()
        # tmp = tmp.dropna()
        df = pd.concat([df,tmp])

del tmp
df.sort_index(inplace=True)